In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import copy
import re
import os
from pathlib import Path
from itertools import chain
from functools import partial

import blpapi
# import clarion
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import DateOffset, BDay
from pandas.tseries.frequencies import to_offset
from plotly import graph_objects as go, express as px
from plotly.subplots import make_subplots
# from clarion import positions

from api.gui.base import View, Tabs, Store
from api.utils import parse_offset
from api.gui.market_monitor_h import CurvesViewer, Pivot, get_issuers_curves, get_bond_ref, get_bday, get_history_table
from api.gui.max_diff_h import LowestFieldCurve
from api.gui.visual_h import Visual
from api.gui.fwd_pair_h import Forward
from api.gui.fwd_pivot_h import PivotForward
from api.blpw import BlpQuery
from api.bbg import BBGField, BBGOverrides
from api.data.base import data_path, tables
from api.data.curves import curves
from api.data.utils import add_pcs, parse_tenor
from api.pricing.cds import get_isda_cds
from api.data.processing import (
    apply_function, 
    add_years_to_maturity, 
    add_maturity_bucket,
    add_custom_rating,
    add_rating_bucket,
    add_geo_area
)


In [3]:
dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

In [4]:
bq = BlpQuery(timeout=50000).start()

In [5]:
br = tables.bonds_reference
bh = pd.read_csv(data_path / 'bonds_history_all.csv', parse_dates=['date'], dayfirst=True)

In [6]:
store = Store()
store.data['bond_ref'] = get_bond_ref()
store.data['bond_hist'] = copy.deepcopy(bh)
# store.data['issuers_curves'] = copy.deepcopy(issuers_curves)

In [ ]:
app = Tabs(
    {
        'curves': CurvesViewer(build=False),
        'pivot': Pivot(build=False),
        'max diff': LowestFieldCurve(build=False),
        'visual': Visual(build=False),
        'fwd pair': Forward(build=False),
        'fwd pivot': PivotForward(build=False)
    },
    store=store
)
app

In [ ]:
from api.gui.market_monitor import get_issuer_curve

In [ ]:
issuer = 'TURKEY'
currency = 'USD'
pcs = 'BGN'
unique = False
field = 'last_price'

In [ ]:
dt_range = pd.date_range(d1, d2, freq='B')
bond_ref_issuer = bond_ref.query(f"ticker == '{issuer}'")
_threshold = 60
high_price_threshold = 150
fld_lbound = None
fld_ubound = None
below_par_threshold = 0.1
above_par_threshold = 0.2

In [19]:
d1 = pd.to_datetime('2020-01-01')
d2 = pd.to_datetime('2022-10-17')

In [18]:
maturity_thresholds = {
    2: 0.5,
    3: 1,
    5: 1,
    7: 1.5,
    10: 3.5,
    20: 3,
    30: 7,
    50: 10,
}


In [22]:
maturities = [5, 10]

In [10]:
bond_ref = store.data['bond_ref']

In [76]:
issuer_securities = [*bond_ref.query(f'ticker == "{issuer}"')['security']]
bh_pcs_issuer = bh.query(f"source =='{pcs}' and security in {issuer_securities}")

In [82]:
method = 'high'

In [91]:
attr

'idxmax'

In [92]:
res = {}
for dt in dt_range:
    br_dt = get_bond_ref(date=dt)
    ic_dt = get_issuer_curve(br_dt, issuer=issuer, maturities=maturities, currency=currency, thresholds=maturity_thresholds, unique=unique)
    mkt_dt = bh_pcs_issuer.query(f"security in {[*ic_dt['security']]} and date == '{dt}'")
    ic_mkt_dt = ic_dt.merge(mkt_dt, on='security', how='left')
    
    fld_lbound_ = fld_lbound if fld_lbound is not None else ic_mkt_dt[field].min()
    fld_ubound_ = fld_ubound if fld_ubound is not None else ic_mkt_dt[field].max()
    ic_mkt_dt = ic_mkt_dt.loc[ic_mkt_dt[field].between(fld_lbound_, fld_ubound_)]
    for index, group in ic_mkt_dt.groupby('maturity_label'):
        attr = 'idxmax' if method == 'high' else 'idxmin'
        sec = group.loc[getattr(group[field], attr)()].to_frame().T
        res[dt, index] = sec

In [94]:
res = pd.concat(res)

In [202]:
res.loc['2021-10':][18:50]

security  ticker                    name          isin  \
2021-10-14 5  0  ZO820002 Corp  TURKEY   TURKEY 6 3/8 10/14/25  US900123CZ18   
           10 7  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-15 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-18 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-19 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-20 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-21 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-22 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-25 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-26 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-27 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-28 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-29 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-11-01 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-11-02 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-11-03 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-11-04 5  3  AM205932 Corp  TURKEY       TURKEY 6 03/25/27  US900123CL22   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   

                currency country bics_sector bics_industry amount_issued  \
2021-10-14 5  0      USD      TR  Government    Sovereigns  2500000000.0   
           10 7      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-15 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10 6      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-18 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10 6      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-19 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10 6      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-20 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10 6      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-21 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10 6      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-22 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10 6      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-25 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10 6      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-26 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10

In [189]:
group_.loc[(dt_, mat_lab)]

KeyError: Timestamp('2020-01-02 00:00:00')

In [205]:
prev = None
res_ = []
broken = False
for dt_, group in res.groupby(level=0, sort=False):
    if prev is not None:

        for i, (mat_lab, group_) in enumerate(pd.concat([prev, group]).groupby(level=1, sort=False)):
            group_ = group_.drop_duplicates(subset='name')
            if len(group_) > 1:
                p_prev = bh_pcs_issuer.query(f'security == "{group_["security"].iloc[0]}" and date == "{dt_}"')[field].squeeze()
                if not isinstance(p_prev, pd.Series):  # implies it's empty and the bond is not present anymore as expired  
                    p_now = group_[field].iloc[1]
                    pct_change = 1 - p_prev / p_now
                    switch = False
                    if method == 'high':
                        if p_now > 100:
                            if pct_change < - above_par_threshold:
                                switch = True
                        else:
                            if pct_change < - below_par_threshold:
                                switch = True
                    else:
                        if p_now > 100:
                            if pct_change > above_par_threshold:
                                switch = True
                        else:
                            if pct_change > below_par_threshold:
                                switch = True                
                    if not switch:
                        group.iloc[i, :] = prev.iloc[i, :]
                        group.iloc[i, group.columns.get_loc(field)] = p_prev
    res_.append(group)
    prev = group

In [206]:
pd.concat(res_).loc['2021-10':][18:50]

security  ticker                    name          isin  \
2021-10-14 5  0  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 7  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-15 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-18 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-19 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-20 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-21 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-22 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-25 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-26 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-27 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-28 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-10-29 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-11-01 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-11-02 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-11-03 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   
2021-11-04 5  3  ED777753 Corp  TURKEY   TURKEY 7 3/8 02/05/25  US900123AW05   
           10 6  EC214944 Corp  TURKEY  TURKEY 11 7/8 01/15/30  US900123AL40   

                currency country bics_sector bics_industry amount_issued  \
2021-10-14 5  0      USD      TR  Government    Sovereigns  3250000000.0   
           10 7      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-15 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10 6      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-18 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10 6      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-19 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10 6      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-20 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10 6      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-21 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10 6      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-22 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10 6      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-25 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10 6      USD      TR  Government    Sovereigns  1500000000.0   
2021-10-26 5  3      USD      TR  Government    Sovereigns  3250000000.0   
           10

In [ ]:
ic_dt

In [ ]:
br_dt.query('ticker == "TURKEY"')

In [ ]:
maturities

In [ ]:
ic_dt